## Problem 1

Find the absolute minimum of the function

\begin{equation}
    f(x) = f(x_1, x_2) = x_1 e^{-{x_1}^2 -{x_2}^2}
\end{equation}

in the domain $x \in \mathbb{R}^2$ (unconstrained problem).

The initial point is $x^0 = (-0.6, -0.3)$.

In [3]:
## Newton's method (2nd order)

import sympy as sp
import numpy as np
import pandas as pd
from sympy.utilities.lambdify import lambdify

# Define the Newton method for optimization
def Newton(f, x0, ε, J):
    # Define symbolic variables x1 and x2
    x1, x2 = sp.symbols('x1 x2')

    # Calculate the gradient of the objective function
    grad_f = sp.Matrix([sp.diff(f, x1), sp.diff(f, x2)])

    # Calculate the Hessian matrix (second derivative) of the objective function
    Hess_f = sp.Matrix([[sp.diff(grad_f[i], x1), sp.diff(grad_f[i], x2)] for i in range(2)])

    # Create lambda functions for numerical evaluation from symbolic expressions
    f_lambda = lambdify((x1, x2), f, 'numpy')  # Objective function
    grad_lambda = lambdify((x1, x2), grad_f, 'numpy')  # Gradient
    hessian_lambda = lambdify((x1, x2), Hess_f, 'numpy')  # Hessian

    # Initialize the parameter vector x with the initial point x0
    x = np.array(x0, dtype=float)
    num_iterations = 0  # Iteration counter

    # List to store optimization data at each iteration
    optimization_data = []

    # Start of the optimization loop
    while num_iterations <= J:
        # Calculate the gradient and Hessian at the current point
        g = np.array(grad_lambda(x[0], x[1]), dtype=float)
        H = np.array(hessian_lambda(x[0], x[1]), dtype=float)

        grad_norm = np.linalg.norm(g)
        fx = f_lambda(x[0], x[1])

        # Append data to the list as a dictionary, separating x into x1 and x2
        optimization_data.append({'Iteration': num_iterations + 0, 'x1': x[0], 'x2': x[1], 'f(x)': fx, '|∇f(x)|': grad_norm})

        if grad_norm <= ε or num_iterations == J:
            break

        # Update the parameter vector x using the Newton method
        x = x - np.reshape(np.dot(np.linalg.inv(H), g), x.shape)
        num_iterations += 1

    # Create a DataFrame to store optimization data
    optimization_data_df = pd.DataFrame(optimization_data)

    # Print the number of iterations and the final result
    print("Iterations performed:", num_iterations)
    print("The point that minimizes the function f is x* = [" + str(x[0]) + ', ' + str(x[1]) + '], such that f(x*) = ' + str(fx))

    return fx, optimization_data_df

# Define symbolic variables x1 and x2
x1, x2 = sp.symbols('x1 x2')

# Define the objective function f(x1, x2)
def f(x1_val, x2_val):
    return x1_val * sp.exp(-x1_val**2 - x2_val**2)

# Set tolerance ε, the max number of iterations J and the initial point x0
ε = 1e-15
J = 100
x0 = [-0.6, -0.3]

# Call the Newton method for optimization
result, optimization_data = Newton(f(x1, x2), x0, ε, J)

# Print the DataFrame with all tne info
print('\n')
print(optimization_data)


Iterations performed: 5
The point that minimizes the function f is x* = [-0.7071067811865476, 0.0], such that f(x*) = -0.42888194248035333


   Iteration        x1            x2      f(x)       |∇f(x)|
0          0 -0.600000 -3.000000e-01 -0.382577  2.908032e-01
1          1 -0.726126  8.738739e-02 -0.425314  8.090296e-02
2          2 -0.706530 -1.485944e-03 -0.428881  1.613678e-03
3          3 -0.707107  8.538448e-09 -0.428882  3.980727e-07
4          4 -0.707107 -1.837995e-21 -0.428882  6.528111e-14
5          5 -0.707107  0.000000e+00 -0.428882  1.110223e-16


The optimal point is found at $x^* = (-0.7071, 0) \rightarrow (-1/\sqrt 2, 0)$ with the minimum value $f(x^*) = -0.4289 \rightarrow -1/\sqrt {2e}$.